In [1]:
import os
import h5py
import numpy as np
import time

start = time.time()
input_datafile = h5py.File('../input/uploads/uploads/tryp')

train_norm_images = np.array(input_datafile['train_norm'])
train_trypo_images = np.array(input_datafile['train_trypo'])
valid_norm_images = np.array(input_datafile['valid_norm'])
valid_trypo_images = np.array(input_datafile['valid_trypo'])

end = time.time()
print(end-start)

1.993772029876709


In [2]:
import numpy as np
from keras.utils import np_utils
    
def process_images(image):
    return np.divide(image, 224)

def join_data(norm, trypo):
    X = np.concatenate([norm, trypo])
    Y1 = np.zeros(len(norm))
    Y2 = np.ones(len(trypo))
    Y = np.concatenate([Y1, Y2])
    Y = np_utils.to_categorical(Y) 
    print(Y.shape)
    print(X.shape)
    return X, Y

train_norm_images = process_images(train_norm_images)
train_trypo_images = process_images(train_trypo_images)
valid_norm_images = process_images(valid_norm_images)
valid_trypo_images = process_images(valid_trypo_images)

print(train_norm_images[0])

X, Y = join_data(train_norm_images, train_trypo_images)
X_val, Y_val = join_data(valid_norm_images, valid_trypo_images)


Using TensorFlow backend.


[[[ 0.94642857  0.75        0.00446429]
  [ 0.94642857  0.75        0.00446429]
  [ 0.94642857  0.75        0.00446429]
  ..., 
  [ 0.96428571  0.72767857  0.        ]
  [ 0.99107143  0.74107143  0.        ]
  [ 0.97321429  0.72321429  0.        ]]

 [[ 0.94642857  0.75        0.00446429]
  [ 0.94642857  0.75        0.00446429]
  [ 0.94642857  0.75        0.00446429]
  ..., 
  [ 0.94642857  0.71875     0.00892857]
  [ 0.96875     0.72767857  0.00892857]
  [ 0.94196429  0.69642857  0.00446429]]

 [[ 0.94196429  0.74553571  0.        ]
  [ 0.94196429  0.74553571  0.        ]
  [ 0.94196429  0.74553571  0.        ]
  ..., 
  [ 0.93303571  0.71428571  0.04017857]
  [ 0.95535714  0.72767857  0.03571429]
  [ 0.92410714  0.69196429  0.00892857]]

 ..., 
 [[ 0.55803571  0.33928571  0.02678571]
  [ 0.55803571  0.33928571  0.02678571]
  [ 0.55803571  0.33482143  0.02678571]
  ..., 
  [ 0.84821429  0.60714286  0.        ]
  [ 0.84821429  0.60714286  0.        ]
  [ 0.84821429  0.61160714  0.     

In [71]:
import keras
from keras.models import Model, Sequential
from keras.layers import GlobalAveragePooling2D, Dense
from keras.applications.vgg16 import VGG16

model = VGG16(include_top = False, weights='imagenet')

def cut_model(n):
    x = GlobalAveragePooling2D()(model.layers[-n-1].output)
    pred = Dense(2, activation='softmax')(x)
    new_model = Model(input = model.input, output = [pred])
    for layer in new_model.layers[:-2]:
        layer.trainable = False
    new_model.compile(optimizer = "adam", loss = "categorical_crossentropy",
                 metrics = ["accuracy"])
    new_model.evaluate(X_val, Y_val)
    return new_model


In [72]:
full_model = cut_model(0)
first_cut_model = cut_model(4)
second_cut_model = cut_model(8)
third_cut_model = cut_model(12)
last_cut_model = cut_model(15)


/usr/local/share/jupyter/kernels/neptune-kernel/neptunekernel.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`
  # Unless required by applicable law or agreed to in writing, software


1000/1000 [==============================] - 5s     


In [ ]:
full_model_hist = full_model.fit(X, Y, epochs = 10, validation_data=[X_val, Y_val])

Train on 15884 samples, validate on 1000 samples
Epoch 1/10
15884/15884 [==============================] - 298s - loss: 0.3665 - acc: 0.8454 - val_loss: 0.2912 - val_acc: 0.8860
Epoch 2/10
12096/15884 [=====================>........] - ETA: 66s - loss: 0.2792 - acc: 0.8903

In [ ]:
print(full_model_hist.history)